In [1]:
import os
%load_ext autoreload
%autoreload 2

from FlexivPy.joy import XBoxController
joy = XBoxController(0)
joy.getStates()

pygame 2.6.1 (SDL 2.28.4, Python 3.10.16)
Hello from the pygame community. https://www.pygame.org/contribute.html
Put your stick at reset and do not touch it while calibrating


{'left_joy': array([ 0., -0.]),
 'right_joy': array([ 0., -0.]),
 'left_trigger': 0.0004999999999999449,
 'right_trigger': 0.0004999999999999449,
 'A': 0,
 'B': 0,
 'X': 0,
 'Y': 0,
 'left_bumper': 0,
 'right_bumper': 0,
 'options_left': 0,
 'options_right': 0,
 'left_joy_btn': 0,
 'right_joy_btn': 0}

Before going through the next cells, start the asynchronous simulator or the real robot bridge by tunning the following in your terminal:

**Asynchronous Simulator:**

```bash
flexivpy_async_sim --mode velocity
```

**Real Robot Client**

```bash
robot_server -cm 3 -g --path /home/FlexivPy/FlexivPy/assets/ -rcf /home/FlexivPy/flexivpy_bridge/config.yaml
```
The `cm` 3 means that the robot is started in joint velocity mode.

In [49]:
import pinocchio as pin
import numpy as np
from FlexivPy.robot.model.pinocchio import FlexivModel
from FlexivPy.robot.interface import FlexivDDSClient
import time
from FlexivPy.robot.dds.subscriber import get_last_msg
import cv2
from cyclonedds.domain import DomainParticipant
from cyclonedds.topic import Topic
from cyclonedds.sub import Subscriber, DataReader
import time

from cyclonedds.domain import DomainParticipant
from cyclonedds.topic import Topic
from FlexivPy.robot.dds.flexiv_messages import EnvImage
from datetime import datetime
import matplotlib.pyplot as plt


class CameraDDSClient:
    def __init__(self):

        self.domain_participant = DomainParticipant(10)

        self.topic_env_image = Topic(self.domain_participant, "EnvImage", EnvImage)
        self.subscriber_env_image = Subscriber(self.domain_participant)
        self.reader_env_image = DataReader(
            self.subscriber_env_image, self.topic_env_image
        )

        self.warning_no_env_image_dt = 0.2
        self.warning_no_env_image_send = 0.2

        self.max_wait_time_first_msg = 2

        

        # create a smiluation in another process
        self.last_img = None

        tic = time.time()
       
        self.time_last_img = tic

        tic = time.time()
        print("waiting to receive the image ...")
        while True:
            if self.is_ready():
                print("Camera is ready!")
                break
            if time.time() - tic > self.max_wait_time_first_msg:
                raise Exception("Camera is not ready! -- Start the server first!")
            time.sleep(.1)

    def is_ready(self):
        return self.get_env_image() is not None


    def get_env_image(self):
        tic = time.time()
        msg = get_last_msg(self.reader_env_image, EnvImage)
        if msg:
            now = datetime.now()  # TODO: avoid calling tic twice
            # print(now.strftime("%Y-%m-%d %H:%M:%S.%f")[:-4])
            # print("msg.time", msg.timestamp)
            np_array = np.frombuffer(bytes(msg.data), dtype=np.uint8)
            frame = cv2.imdecode(np_array, cv2.IMREAD_COLOR)
            self.last_img = frame
            self.time_last_img = tic
            self.warning_no_env_image_send = False
        else:
            if (
                tic - self.time_last_img > self.warning_no_env_image_dt
                and not self.warning_no_env_image_send
            ):
                print(
                    f"warning: client did not recieve env image in  last {self.warning_no_env_image_dt} [s]"
                )
                self.warning_no_env_image_send = True
                self.warning_no_env_image_send = False

        return self.last_img


model = FlexivModel()
robot = FlexivDDSClient()
# Run this in another terminal!
# python FlexivPy/async_camera_app.py

# camera = CameraDDSClient()
# image = camera.get_env_image()
# print(type(image))
# plt.imshow(image)

waiting to receive the first message from the robot...
Robot is ready!


In [84]:
while True:
    time.sleep(0.02)
    print(robot.get_robot_state().ft_sensor)

[-2.915314197540283, 2.529203414916992, 12.1522216796875, -0.03940794616937637, 0.1286187320947647, 0.015385827049612999]
[-2.859004259109497, 2.5613949298858643, 12.158830642700195, -0.03512556478381157, 0.13613459467887878, 0.016471408307552338]
[-2.578059673309326, 2.742306709289551, 12.224252700805664, -0.03225472941994667, 0.14333412051200867, 0.021623000502586365]
[-2.8753035068511963, 2.654676675796509, 12.0380277633667, -0.03331741690635681, 0.1308666616678238, 0.016415031626820564]
[-2.8765690326690674, 2.614387273788452, 12.193870544433594, -0.03506004065275192, 0.13681766390800476, 0.016023950651288033]
[-2.7889530658721924, 2.5644004344940186, 12.060873031616211, -0.03808889910578728, 0.13015805184841156, 0.013063798658549786]
[-2.8532114028930664, 2.6414284706115723, 12.08142375946045, -0.03388633206486702, 0.13523070514202118, 0.020596278831362724]
[-2.8532633781433105, 2.6644036769866943, 12.045761108398438, -0.03448370844125748, 0.14643637835979462, 0.017791327089071274

KeyboardInterrupt: 

ALSA lib pcm.c:8772:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8772:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8772:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8772:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8772:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8772:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8772:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8772:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8772:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8772:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8772:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8772:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8772:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8772:(snd_pcm_recover) underrun occurred


In [34]:
from FlexivPy.robot.dds.flexiv_messages import FlexivCmd
while True:
    joy_state = joy.getStates()
    close = joy_state['A']
    cmd = FlexivCmd()
    if close:
        cmd.g_cmd='close'
        cmd.g_cmd='open'
        cmd.g_width = 0.0
        cmd.g_force = 10
    else:
        cmd.g_cmd='open'
        cmd.g_width = 0.05
        cmd.g_force = 10
    cmd.mode = 3
    robot.set_cmd(cmd)
    time.sleep(0.01)
# robot.get_robot_state().g_width

KeyboardInterrupt: 

In [35]:
from FlexivPy.controllers.jointspace import RRTController, GoJointConfigurationVelocity
from FlexivPy.controllers.runners import NonBlockingRunner, blocking_runner

homing_controller = RRTController(control_mode='velocity')
homing_controller = GoJointConfigurationVelocity(
qgoal = np.array([-0.007916312664747238,
 -0.5759644508361816,
 0.01036121603101492,
 1.4920966625213623,
 -0.006107804365456104,
 0.49659037590026855,
 0.004393362440168858])
)

blocking_runner(robot, homing_controller)

goal reached


<ControllerStatus.GOAL_REACHED: 1>

In [ ]:
import matplotlib.pyplot as plt

import cv2
import numpy as np


def get_small_image(image):

    # Assuming 'image' is your NumPy array image
    # Note: OpenCV uses width before height
    # new_width = int(image.shape[1] / 4 )
    # new_height = int( image.shape[0]  /  4 )
    # new_dimensions = (new_width, new_height)
    # print("new dimensions ", new_dimensions)

    # # Resize the image with INTER_AREA interpolation (best for shrinking)
    # image_resized = cv2.resize(image, new_dimensions, interpolation=cv2.INTER_AREA)


    # # Extract the central 64x64 pixels
    # height, width = image_resized.shape[:2]
    # central_size = 64
    # half_size = central_size // 2

    # center_y = height // 2
    # center_x = width // 2

    # start_y = max(0, center_y - half_size)
    # start_y = 2
    # start_x = max(0, center_x - half_size)
    # end_y = start_y + central_size
    # end_x = start_x + central_size

    # end_y = min(height, end_y)
    # end_x = min(width, end_x)


    # print("start x", start_x)
    # print("start y", start_y)

    start_y = 10
    end_y = -20
    start_x = 20
    end_x = -15


    interest_area = image[start_y:end_y, start_x:end_x]
    # print("interest area" , interest_area.shape)
    interest_area_6464 = cv2.resize(interest_area, (64,64), interpolation=cv2.INTER_AREA)
    return interest_area_6464 , interest_area


image = camera.get_env_image()
small_image , interest_area =  get_small_image(image)


fig, ax = plt.subplots(1,3)


# convert the i
ax[0].imshow(image)
ax[1].imshow(interest_area)
ax[2].imshow(small_image)


In [46]:
from FlexivPy.controllers.taskspace import DiffIKController
import imageio
import pickle
import numpy as np
from PIL import Image
import pathlib

from datetime import datetime
import json
from concurrent.futures import ThreadPoolExecutor
from zoneinfo import ZoneInfo
from FlexivPy.controllers.runners import NonBlockingRunner, blocking_runner
from scipy.spatial.transform import Rotation as Rot
max_v = 0.35

class commandInterface:
    def __init__(self, v_max = 0.2, omega_max = 3.):
        self.v_max = 0.1
        self.omega_max = 30.2
        self.setCommand(np.zeros(6))
        
    def setCommand(self, action):
        self.delta_rot = action[3:]
        self.delta_trans = action[:3]
        t_exec_rot = np.linalg.norm(self.delta_rot)/self.omega_max
        t_exec_trans = np.linalg.norm(self.delta_trans)/self.v_max
        self.t_exec = max(t_exec_rot, t_exec_trans)

        state=robot.get_robot_state()
        info = model.getInfo(np.array(state.q), np.array(state.dq))
        T_start = info["poses"]["link7"].copy()
        self.R_start = T_start[:3,:3]
        self.trans_start = T_start[:3, -1]
        self.start_time = time.time()
        self.end_time = self.start_time + self.t_exec

    def computeCommand(self, t):
        t = np.clip(t, self.start_time, self.end_time)
        if self.end_time > self.start_time:
            alpha = ((t-self.start_time)/(self.end_time - self.start_time))
        else:
            alpha = 1.0
        t_cmd = self.trans_start + alpha*self.delta_trans
        theta_cmd = alpha*self.delta_rot
        R_cmd = Rot.from_euler('xyz', theta_cmd).as_matrix() @ self.R_start
        return t_cmd, R_cmd

    def __call__(self, state):
        t_cmd, R_cmd = self.computeCommand(time.time())
        T_cmd = np.eye(4)
        T_cmd[:3, :3] = R_cmd
        T_cmd[:3, -1] = t_cmd
        return T_cmd


command_interface = commandInterface()

task_controller = DiffIKController(
    model,
    T_cmd_fun=command_interface,
    dt=0.01,
    dq_max=20.0,
    max_error = 0.2,
    kp = 1.5 * np.diag([10.0, 10.0, 10.0, 10.0, 10.0, 10.0]),
    control_mode="velocity",
)

def save_image_pil(im, fout):
    # A helper function to save the image
    im.save(fout)


class CallbackFun:
    def __init__(self, img_dir, img_dir_big=None):
        self.cmds = []
        self.callback_dt = 0.1
        self.states = []
        self.img_names = []
        self.last_callback_t = -1
        self.img_counter = 0
        self.img_dir = img_dir
        self.img_dir_big = img_dir_big

        # Create a thread pool executor for asynchronous saving
        self.executor = ThreadPoolExecutor(max_workers=1)

    def __call__(self, robot, cmd, _):
        tic = time.time()
        if tic - self.last_callback_t > self.callback_dt:
            self.last_callback_t = tic
            self.cmds.append(cmd)
            self.states.append(robot.get_robot_state())

            # store the image as png.
            # img = camera.get_env_image()

            # small_image, interest_area = get_small_image(img)

            # Convert images to PIL
            # im_small = Image.fromarray(small_image)
            # fout_small = self.img_dir / f"img_{self.img_counter:09d}.png"

            # Submit the small image saving task to thread
            # self.executor.submit(save_image_pil, im_small, fout_small)

            # self.img_names.append(str(fout_small))

            # if self.img_dir_big is not None:
                # im_big = Image.fromarray(img)
                # fout_big = self.img_dir_big / f"img_{self.img_counter:09d}.png"

                # Submit the big image saving task to thread
                # self.executor.submit(save_image_pil, im_big, fout_big)

            self.img_counter += 1

command_interface.setCommand(np.array([0, 0, -0., 0, 0, 0]))
status = NonBlockingRunner(robot, task_controller, timeout=30 * 60)

In [ ]:
status.close()

ALSA lib pcm.c:8772:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8772:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8772:(snd_pcm_recover) underrun occurred


In [47]:
task_controller.gripper_width=0.05

In [28]:

for i in range(1000):
    time.sleep(0.2)
    joy_state=joy.getStates()
    left = joy_state['left_joy']
    command_interface.setCommand(np.array([0, 0, 0.0, 3*left[0]*0.1, 3*left[1]*0.1, 0.0]))

KeyboardInterrupt: 

In [33]:
command_interface.setCommand(np.array([0, 0, 0.0, 0.5, 0., 0.0]))

In [ ]:
from scipy.spatial.transform import Rotation as R
from scipy.spatial.transform import Slerp

rot1 = R.from_euler('xyz', np.array([0, 0, 0]))
rot2 = R.from_euler('xyz', np.array([0, 0, 2]))
Slerp()

In [798]:
get_T_from_controller_no_drift(robot.get_robot_state())[:3,-1]

array([ 0.70192113, -0.11228298,  0.52460145])

In [4]:
robot.get_robot_state().q

[-0.007947116158902645,
 -0.5770066976547241,
 0.013467387296259403,
 1.4921327829360962,
 -0.006515359506011009,
 0.499993234872818,
 0.006887102499604225]

In [ ]:
print(len(callback.img_names))
# lets create a video. 
                        
import pickle                            
import numpy as np
from PIL import Image
import pathlib


# imageio.mimsave("tmp.mp4" , np.stack(callback.imgs) , fps=10., codec = "h264")

# save imgs as png.

# img_dir = pathlib.Path("data_t_v0") / folder_name / "img/"
data_dir =  pathlib.Path("data_t_v0") / folder_name / "data/"


print("data_dir ", data_dir )

# for i, img in enumerate(callback.imgs) : 
#     #
#     im = Image.fromarray(img)
#     fout = img_dir / f"img_{i:09d}.png"
#     img_names.append(str(fout))
#     im.save(fout)

Dataout = {
            "q": [ [i for i in x.q ] for x in callback.states ] ,
            "dq" :  [ [i for i in x.dq ] for x in callback.states ] ,
            "imgs" : callback.img_names
        }

with open(data_dir / "data.pkl", "wb") as f:
    pickle.dump(
        Dataout , f
    )

with open(data_dir / "data.json", "w") as f:
    json.dump(
        Dataout
        , f
    )



In [ ]:
# quick prototype of getting images directly with open cv. not well tested. 

import cv2
import threading
import matplotlib.pyplot as plt
import time
class QVideoCapture:
    def __init__(self, name):
        self.cap = cv2.VideoCapture(name)
        self.lock = threading.Lock()
        self.running = True  # Flag to control the thread
        self.t = threading.Thread(target=self._reader)
        self.t.start()

    # Grab frames as soon as they are available
    def _reader(self):
        while self.running:
            with self.lock:
                ret = self.cap.grab()
                time.sleep(0.01)
                # sleep 
            if not ret:
                # If grabbing fails, perhaps the camera was disconnected
                break

    # Retrieve the latest frame
    def read(self):
        with self.lock:
            ret, frame = self.cap.retrieve()
        return ret, frame

    def release(self):
        self.running = False
        self.t.join()
        self.cap.release()


cap = QVideoCapture(5)
ret,frame = cap.read()

fig, ax = plt.subplots(1,3)
cap.release()

# convert the i
ax[0].imshow(frame)

